# Training a Machine Learning model for the MNIST Handwritten digits dataset

## 1. Introduction

MNIST ("Modified National Institute of Standards and Technology") Handwritten Digit recogition is often regarded as the "Hello World" of Machine Learning. This project involvs creating a Machine Learning model using a simple Convolutional Neural Network to further enhance the capabilities of the model in recognising the Handwritten Digits. We then run and test the model on user inputs.

We begin by: 
**Importing the required libraries**


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.models import Sequential, Model
from keras.layers import Conv2D, Lambda, MaxPooling2D # convolution layers
from keras.layers import Dense, Dropout, Flatten # core layers

from keras.layers.normalization import BatchNormalization

from keras.preprocessing.image import ImageDataGenerator

from keras.utils.np_utils import to_categorical

## 2. Data Preprocessing

### 2.1 Importing the datasets

I found a dataset that was already split into training set and testing set, both are subsets of the MNIST Handwritten digits dataset

In [ ]:
import os
train = pd.read_csv('../input/mnist-in-csv/mnist_train.csv')
test = pd.read_csv('../input/mnist-in-csv/mnist_test.csv')
print("Data Ready")

In [ ]:
print(f"Training data size is {train.shape}\nTesting data size is {test.shape}")

**Set data features and labels**


In [ ]:
test

In [ ]:
X = train.drop(['label'], 1).values
y = train['label'].values
test_x = test.drop(['label'],1).values

### 2.2 Normalization
We perform a grayscale normalization which reduces the time as well as avoids looking for useless details in the image

In [ ]:
X = X / 255.0
test_x = test_x / 255.0

### 2.3 Reshaping
Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1) 

canal = 1:  For gray scale

In [ ]:
X = X.reshape(-1,28,28,1)
test_x = test_x.reshape(-1,28,28,1)

### 2.4 Label Encoding
**Encode labels to one-hot-vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])**

In [ ]:
y = to_categorical(y)


### 2.5 Split training and valdiation set
The validation set is to evaluate the performance of the model


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

**Visualizing the Data**

In [ ]:
X_train__ = X_train.reshape(X_train.shape[0], 28, 28)

fig, axis = plt.subplots(1, 4, figsize=(20, 10))
for i, ax in enumerate(axis.flat):
    ax.imshow(X_train__[i], cmap='binary')
    digit = y_train[i].argmax()
    ax.set(title = f"Real Number is {digit}");

**Normalization**


In [ ]:
mean = np.mean(X_train)
std = np.std(X_train)

def standardize(x):
    return (x-mean)/std

In [ ]:
epochs = 50
batch_size = 64

# 3. Convolutional Neural Network

## 3.1 Defining the model

This uses a Keras Sequential API to create a CNN, we start from the input and add the layers one by one.

The first layer is the convolutional (Conv2D) layer. We choose 32 filters for the first-two conv2D layers and 64 filters for the second-two layers and 128 filters for third-two layers and 256 for the last one. Each filter transforms a part of the image as defined by the kernel size using the kernel filter. The kernel filter matrix is applied on the whole image. Filters can be seen as a transformation of the image.

The CNN can isolate features that are useful everywhere from these transformed images (feature maps).

The second important layer in CNN is the pooling (MaxPool2D) layer. This layer simply acts as a downsampling filter. It looks at the 2 neighboring pixels and picks the maximal value. These are used to reduce computational cost, and to some extent also reduce overfitting.

Combining convolutional and pooling layers, CNN is able to combine local features and learn more global features of the image.

'relu' is the rectifier (activation function max(0,x)). The rectifier activation function is used to add non linearity to the network.

The Flatten layer is used to convert the final feature maps into a single 1D vector. This flattening step is needed so that we can make use of fully connected layers after some convolutional/maxpool layers. It combines all the found local features of the previous convolutional layers.

In the end we use the features in two fully-connected (Dense) layers which is just an artificial neural networks (ANN) classifier. In the last layer(Dense(10,activation="softmax")) the net outputs distribution of probability of each class i.e. [0,0,0.98,0,0,0,0,0,0,0]

**Model Definition**

In [ ]:
model=Sequential()

 
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu", input_shape=(28,28,1)))
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())    
model.add(Conv2D(filters=256, kernel_size = (3,3), activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(512,activation="relu"))

model.add(Dense(10,activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

## Model Summary

In [ ]:
model.summary()

## 3.2 Augmenting the data

This step was necessary because we are dealing with the user input, the user is free to draw anything he wants, so the drawing of the digits might not always be in perfect orientation. Hence we need to train the model for handiling such situations

I found out the following data augmentation techniques that are frequently used by people for further improving the accurary of their models.

The improvement:
 - Without data augmentation, Accuracy:  98.114%
 - With data augmentation, Accuracy:  99.67% 


In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

train_gen = datagen.flow(X_train, y_train, batch_size=batch_size)
test_gen = datagen.flow(X_test, y_test, batch_size=batch_size)

## 3.3 Model Training

In [ ]:
# Fit the model
history = model.fit_generator(train_gen, 
                              epochs = 2, 
                              steps_per_epoch = X_train.shape[0] // batch_size,
                              validation_data = test_gen,
                              validation_steps = X_test.shape[0] // batch_size)

In [ ]:
import keras.backend as K

inputs = model.input
outputs = K.function([inputs], [layer.output for layer in model.layers])



In [ ]:
# Call this function for getting the outputs for each layer
len(outputs([X_train[0:1]]))

In [ ]:
y_pred = model.predict(X_test)
X_test__ = X_test.reshape(X_test.shape[0], 28, 28)

fig, axis = plt.subplots(4, 4, figsize=(12, 14))
for i, ax in enumerate(axis.flat):
    ax.imshow(X_test__[i], cmap='binary')
    ax.set(title = f"Real Number is {y_test[i].argmax()}\nPredict Number is {y_pred[i].argmax()}");

# Saving the trained Model

In [ ]:
model.save("mnist_trained_99.h5")

## Part 1 finishes here(Model Training)

`___________________________________________________________________________________________________________`

# Part 2 

### Using the model to predict the digits

### Loading the model from the file


In [ ]:
from keras.models import load_model
model = load_model("./mnist_trained_99.h5") # Path where the model is saved.
print("Model loaded successfully")

In [ ]:
from skimage import color, io
from skimage.transform import resize

# Using the model to predict our custom handwritten digits

In [ ]:
digits_folder = "../input/handwritten-digits-images"  # Folder containing the images of handwritten digits
images = os.listdir(digits_folder)  # Custom image list


fig, axis = plt.subplots(4, 4, figsize=(12, 14))
for i, ax in enumerate(axis.flat):
    image = io.imread(digits_folder + "/" + images[i])
    ax.imshow(image, cmap='binary')
    image = color.rgb2gray(image)
    image_resized = resize(image, (28, 28, 1))
    final = 1 - np.array(image_resized)
    final = np.expand_dims(final, axis=0)
    answer = model.predict(final)
    ret_val = answer.argmax()
    ax.set(title = f"Predicted number: {ret_val}");
